In [ ]:
import os
from dotenv import load_dotenv
import requests

# 현재 작업 디렉토리 확인 및 프로젝트 루트로 이동
print(f"Current working directory: {os.getcwd()}")
# 노트북에서는 상위 디렉토리로 이동
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
print(f"Changed to: {os.getcwd()}")

load_dotenv()
api_key = os.getenv("UPSTAGE_API_KEY")

input_file = "data/불법스팸_방지를_위한_정보통신망법_안내서(24.3월).pdf"
output_prefix = "output/bulbeobseupaem_bangji_202403"  # output 파일명 앞부분

# 파일 존재 여부 확인
if not os.path.exists(input_file):
    print(f"파일이 존재하지 않습니다: {input_file}")
    print(f"현재 디렉토리의 data 폴더 내용:")
    if os.path.exists("data"):
        print(os.listdir("data"))
    else:
        print("data 폴더가 없습니다")
else:
    print(f"파일 찾음: {input_file}")

with open(input_file, "rb") as f:
    response = requests.post(
        "https://api.upstage.ai/v1/document-digitization",
        headers={"Authorization": f"Bearer {api_key}"},
        files={"document": f},
        data={
            "ocr": "force",
            "coordinates": True,
            "output_formats": '["html", "text"]',
            "model": "document-parse",
        },
    )

result = response.json()

# output 디렉토리 생성
os.makedirs("output", exist_ok=True)

# 추출된 텍스트 저장 (.txt)
with open(f"{output_prefix}.txt", "w", encoding="utf-8") as f:
    f.write(result["content"]["text"])

# 추출된 html 저장 (.md, 원하는 경우)
with open(f"{output_prefix}.html", "w", encoding="utf-8") as f:
    f.write(result["content"]["html"])

from markdownify import markdownify

html_content = result["content"]["html"]
md_content = markdownify(html_content)
with open(f"{output_prefix}.md", "w", encoding="utf-8") as f:
    f.write(md_content)

# 결과 전체를 json으로 저장
import json

with open(f"{output_prefix}.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print("추출 결과를 output 폴더에 저장 완료!")

Current working directory: d:\workdir\github-space-cap\Jober_ai\notebooks
Changed to: d:\workdir\github-space-cap\Jober_ai
파일 찾음: data/불법스팸_방지를_위한_정보통신망법_안내서(24.3월).pdf
추출 결과를 output 폴더에 저장 완료!
